- Id - um Id que representa um duplicado (Loja, Data) dentro do conjunto de teste
- Loja - um Id exclusivo para cada loja
- Vendas - o volume de negócios de um determinado dia (é isso que você está prevendo)
- Clientes - o número de clientes em um determinado dia
- Aberto - um indicador para saber se a loja estava aberta: 0 = fechada, 1 = aberta
- FeriadoEstadual - indica um feriado estadual. Normalmente, todas as lojas, com poucas exceções, fecham em feriados estaduais. Observe que todas as escolas fecham em feriados e fins de semana. a = - feriado, b = Páscoa, c = Natal, 0 = Nenhum
- SchoolHoliday - indica se a (Loja, Data) foi afetada pelo fechamento de escolas públicas
- StoreType  - diferencia entre 4 modelos de loja diferentes: a, b, c, d
- Sortimento - descreve um nível de sortimento: a = básico, b = extra, c = estendido
- CompetitionDistance - distância em metros até a loja concorrente mais próxima
- CompetitionOpenSince[Mês/Ano] - fornece o ano e o mês aproximados da abertura do concorrente mais próximo
- Promoção - indica se uma loja está realizando uma promoção naquele dia
- Promo2 - Promo2 é uma promoção contínua e consecutiva para algumas lojas: 0 = loja não está participando, 1 = loja está participando
- Promo2Since[Ano/Semana] - descreve o ano e a semana do calendário em que a loja começou a participar do Promo2
- PromoInterval - descreve os intervalos consecutivos em que a Promo2 é iniciada, indicando os meses em que a promoção é reiniciada. Por exemplo, "fev, mai, ago, nov" significa que cada rodada começa em fevereiro, maio, agosto, novembro de qualquer ano para aquela loja.

# Questão do Negócio
- CFO da empresa fez uma reunião com todos os gerentes da loja e pediu para que cada um deles trouxesse uma previsão diária das proximas 6 semanas de vendas.

- Depois dessa reunião, todos os gerentes entraram em contato, solicitando uma previsão de vendas de sua loja.

1. Qual a motivação:
- A previsão de vendas foi requisitada pela CFO em uma reunião mensal sobre os resultados das lojas

2. Causa raiz do problema
- Dificuldade em determinar valor assertivo para reforma de cada loja

3. Dono do problema:
- CFO

4. Formato da solução
- Granularidade:
    - previsão de vendas por dia nas lojas para os proximos, 42 dias, 6 semanas.
- Tipo do problema:
    - previsão de vendas
- Potenciais métodos:
    - séries temporais
- Formato da entrega:
    - O valor das vendas no final de 6 semanas
    - Como essas vendas acontecem diariamente
    - visualizar com o código da loja o valor ao final da 6 semana


### Ciclos:
- Questão do Negocio
    - entender o stakeholer
- Entendimento do Negócio
    - entender oq precisa ser resolvido
- Coleta de Dados
    - SQL
- Limpeza dos Dados
    - removendo caracteres especiais
    - analisando dtypes
- Exploração de Dados
    - quais variaveis impactam os modelo a ser analisado
    - testes estatísticos
- Modelagem de Dados
    - treino / teste
    - encoder
    - transformação de dados
- Algoritmos de ML
    - aplicar modelos
- Avaliação do algoritmo
    - RMSE
    - MAE
    - MAPE
    - Acurácia
    - Precisão / Recall

# Imports

In [1]:
import pandas as pd
import numpy as np
import inflection
import math

## 0.1 Helper Functions

## 0.2 Load Datas

In [2]:
raw_store = pd.read_csv('rossmann_store_sales/store.csv', low_memory=False)
raw_test = pd.read_csv('rossmann_store_sales/train.csv', low_memory=False)

# merge
df_raw = pd.merge(raw_test, raw_store, how='left', on='Store')

# 1.0 Descrição dos dados

In [3]:
df1 = df_raw.copy()

## 1.1 Rename Columns

In [4]:
#metodo para por em lower e quebrar com "_" antes de maiusculas
snakecase = lambda x: inflection.underscore(x)

#substituindo
df1.columns = list(map(snakecase, df1.columns))

## 1.2 Data Dimensions

In [5]:
print("Numero de linhas:", df1.shape[0])
print("Numero de colunas:", df1.shape[1])

Numero de linhas: 1017209
Numero de colunas: 18


## 1.3 Data Types

In [6]:
df1['date'] = pd.to_datetime(df1['date'])
df1.dtypes

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month           float64
competition_open_since_year            float64
promo2                                   int64
promo2_since_week                      float64
promo2_since_year                      float64
promo_interval                          object
dtype: object

## 1.4 Check NA

In [7]:
df1.isna().sum()

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64

## 1.5 Fillout NA

In [8]:
df1['competition_distance'].max()

np.float64(75860.0)

In [9]:
#substituindo por um outlier
# competition_distance
df1['competition_distance'] = df1['competition_distance'].apply(lambda x: 200000.0 if math.isnan(x) else x)


#usando a coluna da data da ultima venda da loja para preencher os NA
# competition_open_since_month
df1['competition_open_since_month'] = df1['competition_open_since_month'].fillna(df1['date'].dt.month)

# competition_open_since_year
df1['competition_open_since_year'] = df1['competition_open_since_year'].fillna(df1['date'].dt.year)

# promo2_since_week
df1['promo2_since_week'] = df1['promo2_since_week'].fillna(df1['date'].dt.isocalendar().week)

# promo2_since_year
df1['promo2_since_year'] = df1['promo2_since_year'].fillna(df1['date'].dt.year)

# promo_interval
#ideia é: se month_map está em promo_interval existe uma promoção, caso contrario não existe

# criando mes da venda
month_map = {1: 'Jan',
 2: 'Fev',
 3: 'Mar',
 4: 'Apr',
 5: 'May',
 6: 'Jun',
 7: 'Jul',
 8: 'Aug',
 9: 'Sep',
 10:'Oct',
 11:'Nov',
 12:'Dec'}

#nulo por 0
df1['promo_interval'].fillna(0, inplace=True)
#preenchendo
df1['month_map'] = df1['date'].dt.month.map(month_map)

#se atender if´s, tem promo
df1['is_promo'] = df1[['promo_interval', 'month_map']].apply(lambda x: 0 if x['promo_interval'] == 0 else 1 if x['month_map'] in x['promo_interval'].split(',') else 0, axis=1)

C:\Users\MD-user\AppData\Local\Temp\ipykernel_14800\2795566633.py:37: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df1['promo_interval'].fillna(0, inplace=True)


In [12]:
df1.isna().sum()

store                           0
day_of_week                     0
date                            0
sales                           0
customers                       0
open                            0
promo                           0
state_holiday                   0
school_holiday                  0
store_type                      0
assortment                      0
competition_distance            0
competition_open_since_month    0
competition_open_since_year     0
promo2                          0
promo2_since_week               0
promo2_since_year               0
promo_interval                  0
month_map                       0
is_promo                        0
dtype: int64

# 2.0

# 3.0

# 4.0

# 5.0

# 6.0

# 7.0

# 8.0